In [1]:
import impmeas as imp
import pandas as pd
import os
from time import time
from tqdm.notebook import tqdm
import sys; sys.setrecursionlimit(15000)
imp.set_pmc_solver(imp.GPMC())

rpath = "./results/ds_iscas99.csv"
fdir = "./cnfs/iscas99"
fnames = [ f"b{str(idx+1).zfill(2)}.bench.dimacs" for idx in range(13) ]
fpaths = [ f"{fdir}/{fname}" for fname in fnames ]

def execute_method(df, cnf_name, method_name, pbar, func):
    if pbar is not None:
        pbar.set_description(f"{cnf_name} / {method_name}")
    else:
        print(f"{cnf_name} / {method_name}")
    t1 = time() 
    ret = func()
    dt = time() - t1
    df.loc[cnf_name, method_name] = dt
    return ret

def execute_example(df, fpath, methods, pbar):
    fname = fpath.split("/")[-1]
    parsed, cnf, nvars, nclauses = imp.parse_dimacs(open(fpath).read())
    formula = None

    if (m:="BDD_initial_order") in methods: 
        vs = execute_method(df, fname, m, pbar, lambda: imp.representation.random_order(cnf))
        imp.buddy_initialize([f"x{idx}" for idx in vs])
        imp.representation.set_dynamic_reordering()

    if (m:="GPMC_parsing") in methods: formula = execute_method(df, fname, m, pbar, lambda: imp.Formula.parse(parsed))
    if formula is not None:
        if (m:="GPMC_satcount") in methods: execute_method(df, fname, m, pbar, lambda: imp.GPMC().satcount(cnf))
        if (m:="GPMC_influence") in methods: execute_method(df, fname, m, pbar, lambda: imp.influence(formula, "x1"))
        if (m:="GPMC_raw_influence") in methods: execute_method(df, fname, m, pbar, lambda: imp.influence_cnf(cnf, "x1", imp.GPMC()))
    if os.path.getsize(fpath) / 1024 < 10: # only if smaller than 10 kB
        if (m:="BDD_parsing") in methods: bdd = execute_method(df, fname, m, pbar, lambda: imp.BuddyNode.parse(parsed))
        if (m:="BDD_expectation") in methods: execute_method(df, fname, m, pbar, lambda: bdd.expectation())
        if (m:="BDD_influence") in methods: execute_method(df, fname, m, pbar, lambda: imp.influence(bdd, "x1"))
        if (m:="BDD_dcgm") in methods: execute_method(df, fname, m, pbar, lambda: imp.dominating_cgm(bdd))
        if os.path.getsize(fpath) / 1024 < 2:
            if (m:="BDD_blame") in methods: execute_method(df, fname, m, pbar, lambda: imp.blame(bdd, "x1", cutoff=0, debug=True))


In [ ]:
# comment out methods below to actually execute them (note that there are some inter-dependencies, e.g. "GPMC_influence" and "GPMC_parsing")
methods = [
    # "BDD_initial_order",
    # "BDD_parsing",
    # "BDD_expectation",
    # "BDD_influence",
    # "BDD_dcgm",
    # "BDD_blame",
    "GPMC_parsing",
    # "GPMC_satcount",
    # "GPMC_influence",
    "GPMC_raw_influence",
]

df = pd.read_csv(rpath, index_col=0) if os.path.exists(rpath) else pd.DataFrame()
for fpath in (pbar := tqdm(fpaths)):
    execute_example(df, fpath, methods, pbar)
    df.to_csv(rpath)

In [4]:
import pandas as pd

df = pd.read_csv(rpath, index_col=0)
for fpath in fpaths:
    fname = fpath.split("/")[-1]
    lines = open(fpath).read().split("\n")
    for l in lines:
        if l.strip().startswith("p"):
            nvars, nclauses = l.split()[2:]
            df.loc[fname, "nvars"] = int(nvars)
            df.loc[fname, "nclauses"] = int(nclauses)
            break
df.sort_values("nvars")

,BDD_initial_order,GPMC_parsing,GPMC_expectation,GPMC_influence,BDD_parsing,BDD_expectation,BDD_influence,BDD_dcgm,BDD_blame,GPMC_satcount,GPMC_raw_influence,nvars,nclauses
b02.bench.dimacs,0.000015,0.001069,0.065288,0.083172,0.001393,0.000062,0.000256,0.002580,3.649372,0.003209,0.007335,26.0,66.0
b06.bench.dimacs,0.000024,0.000931,0.111390,0.088693,0.003102,0.000152,0.000304,0.006914,697.573153,0.003577,0.007106,44.0,122.0
b01.bench.dimacs,0.000023,0.001006,0.142879,0.122326,0.004020,0.000059,0.000292,0.008528,3068.667951,0.004565,0.009718,45.0,120.0
b03.bench.dimacs,0.000056,0.003852,0.782632,0.448155,52.603779,0.006461,0.024457,1.722537,NaN,0.005717,0.014703,156.0,376.0
b09.bench.dimacs,0.000069,0.003178,1.000671,0.475172,22.952434,0.003777,0.013485,1.156608,NaN,0.010604,0.021204,169.0,417.0
b08.bench.dimacs,0.000080,0.003667,1.196736,0.553596,11.538052,0.006114,0.017282,1.629873,NaN,0.012684,0.025712,180.0,455.0
b10.bench.dimacs,0.000081,0.004637,1.519958,0.651105,18.929197,0.010961,0.045387,3.209551,NaN,0.013562,0.029808,201.0,525.0
b13.bench.dimacs,0.000141,0.006635,3.750168,1.041928,NaN,NaN,NaN,NaN,NaN,0.017276,0.035046,352.0,847.0
b07.bench.dimacs,0.000207,0.008918,7.349442,2.036992,NaN,NaN,NaN,NaN,NaN,0.028616,0.058245,435.0,1132.0
b04.bench.dimacs,0.000295,0.054428,20.422340,4.330727,NaN,NaN,NaN,NaN,NaN,0.162104,0.327203,730.0,1919.0
